# PyConES 2022: Chatbots, Reconocimiento de Voz y Text-to-Speech: Tu Asistente Virtual Open-Source 



## 0. ¿Qué vamos a necesitar?

Necesitaremos Python 3.7.X y las siguientes herramientas que podremos descargar a través de pip

- Rasa
- PyAudio (requiere el paquete de PortAudio en Linux/Mac)
- eSpeakNG (requiere descargarse el programa) 
- Vosk

> **Note**:
> 
> Para el caso de Mac OS, eSpeakNG se puede instalar usando Homebrew o MacPorts, pero si no os funcionara podéis compilar el código fuente
> de esta manera:
> ```bash
> # PCAudioLib (Biblioteca de soporte para eSpeakNG)
> brew install autoconf automake libtool
> git clone https://github.com/espeak-ng/pcaudiolib.git
> cd pcaudiolib
> mv CHANGELOG.md ChangeLog.md
> ./autogen.sh
> ./configure --prefix=/usr/local
> make
> sudo make install
> cd ..
>
># eSpeakNG, ahora sí
> git clone https://github.com/espeak-ng/espeak-ng.git
> cd espeak-ng
> mv CHANGELOG.md ChangeLog.md
> ./configure --exec-prefix=/usr/local/ --datarootdir=/usr/ --sysconfdir=/usr/local --sharedstatedir=/usr/local --localstatedir=/usr/local --includedir=/usr/local --with-extdict-ru --with-extdict-zh --with-extdict-zhy
> make
> sudo make LIBDIR=/usr/local/lib install
> ```


> **Note**:
> 
> En el caso de Windows, una vez se instale eSpeakNG, hay que meter la ruta del ejecutable
> en el PATH, en el apartado "Variables de entorno del sistema" (la ruta sería algo como `C:\Program Files\eSpeak NG\`)

In [ ]:
!pip install rasa
!pip install pyaudio
!pip install espeakng
!pip install vosk

## 1. Manejando el audio desde nuestro PC

Para manejar la grabación y la reproducción de audio en nuestro ordenador a través de Python podemos usar PyAudio.

| Nombre      | PyAudio                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | MIT                                                                                                                                                                                                                                                                                 |
| Descripción | (Traducido) PyAudio proporciona enlaces de Python para PortAudio v19, la biblioteca de E/S de audio multiplataforma. Con PyAudio, puedes utilizar fácilmente Python para reproducir y grabar audio en una variedad de plataformas, como GNU/Linux, Microsoft Windows y Apple macOS. |

Los archivos a grabar/reproducir usan el estándar WAVE para poder extraer toda la información.

### 1.1 Muestreo y precisión
En el mundo del audio digital, todo funciona por 1s y 0s. Pero el sonido es realmente analógico, por lo que hay que parametrizar el mundo analógico en datos discretos. De esta manera, necesitamos recoger con cierta frecuencia la información, y también alguna manera de representar la intensidad de la información del audio.

A la frecuencia de recogida se le conoce como **muestreo**.

A el nivel de detalle que podemos recoger la información del audio en digital se le conoce como **precisión** o **profundidad**. Cuantos más bits se requieran para tomar el dato, más profundidad tendrá.


![Muestreo](img/muestreo.png)
![Precisión](img/precision.png)


In [1]:
# Script de reproducción
import wave
import pyaudio

pyaudio_play = pyaudio.PyAudio()
archivo = wave.open('file.wav','rb')

stream = pyaudio_play.open(
    format = pyaudio_play.get_format_from_width(archivo.getsampwidth()),
    channels = archivo.getnchannels(),
    rate = archivo.getframerate(),
    output = True
)

datos = archivo.readframes(2048)

while len(datos) > 0:
        # writing to the stream is what *actually* plays the sound.
    stream.write(datos)
    datos = archivo.readframes(2048)

pyaudio_play.close(stream)
print("Ha acabado la reproducción")

Ha acabado la reproducción


In [2]:
# Script de grabación
from array import array
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file.wav"
THRESHOLD = 2000
 
audio = pyaudio.PyAudio()
audio_data = []
can_record = False
finish_record = False
silent_chunks = 0
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print ("recording...")
 

while not finish_record:
    chunk = stream.read(CHUNK)
    data_array = array('h', chunk)
    print(max(data_array))
    if not can_record:
        can_record = max(data_array) >= THRESHOLD
    else:
        audio_data.append(chunk)
        if (max(data_array) <= THRESHOLD):
            silent_chunks += 1
            print("Silencio {0}".format(silent_chunks))
        else:
            silent_chunks = 0
        if silent_chunks >= 10:
            finish_record = True
    


print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(audio_data))
waveFile.close()

recording...
1
1
1
4728
3487
3408
10997
12401
9628
9754
17171
19317
17485
14549
32472
32021
18584
14577
22271
25081
18301
5836
14194
14212
8302
24855
32476
25407
29193
27249
26621
27691
20272
10742
16879
15585
295
Silencio 1
549
Silencio 2
86
Silencio 3
80
Silencio 4
52
Silencio 5
134
Silencio 6
123
Silencio 7
107
Silencio 8
71
Silencio 9
81
Silencio 10
finished recording


## 2. Reconocimiento de voz. Convirtiendo el sonido en texto. 

El Reconocimiento de Voz o Automatic Speech Recognition (ASR) como la capacidad de un programa de procesar el habla y convertirlo en un texto escrito legible, que puedan entender también las máquinas para el posterior procesado de la información.

Para reconocer la voz del contenedor del audio, este se extrae en trocitos o chunks que se tratan posteriormente para sacar un texto plano.

En nuestro caso usaremos **Vosk** ya que es muy efectivo y los modelos son muy ligeros.

| Nombre      | Vosk                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | Apache License 2.0                                                                                                                                                                                                                                                                                 |
| Descripción | Vosk es un conjunto de herramientas de reconocimiento de voz con una API de streaming para la mejor experiencia de usuario. También hay bindings para diferentes lenguajes de programación: java/csharp/javascript, etc. Permite una rápida reconfiguración del vocabulario para obtener la mejor precisión.|

Los modelos de Vosk se pueden extraer de [este link](https://alphacephei.com/vosk/models) y para probarlo solo hay que extraer el ZIP y llamar en el código a la carpeta del modelo.

In [3]:
#!/usr/bin/env python3
# Script de reconocimiento.

from vosk import Model, KaldiRecognizer
import sys
import json

model = Model(model_path='model')

# Large vocabulary free form recognition
rec = KaldiRecognizer(model, 16000)

wf = open('file.wav', "rb")
wf.read(44) # skip header

while True:
    data = wf.read(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print('Partial')
        print (res['text'])

res = json.loads(rec.FinalResult())
print('Total')
print (res['text'])

Total
una una una una hora


## 3. Text-to-Speech. Convirtiendo el texto en voz.

Si el reconocimiento de voz trata el procesamiento del habla para extraer un texto, la síntesis de voz sería lo contrario, la producción artificial de ese habla.
Para ello se usan como base los grafemas (aquellas letras y grupos que se pronuncian de una manera). De esa forma, se separa el texto en dichos grafemas, tras lo cual se asocian esos grafemas a sus correspondientes sonidos o fonemas asociados, entonando así cada palabra, frase y finalmente leyendo el texto.

En este caso usaremos [eSpeakNG](https://github.com/espeak-ng/espeak-ng), por su facilidad de uso. Se puede instalar fácilmente en Linux y Windows (y en MacOS se está trabajando en un port). Por contra, la voz no es muy natural.

| Nombre      | eSpeakNG                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | Apache License 2.0                                                                                                                                                                                                                                                                                 |
| Descripción | |

¡Ojo! Para usarse en Python sigue los pasos que indican en el paquete de pip (https://pypi.org/project/espeakng/)



In [5]:
# Script de síntesis de voz
from espeakng import Speaker

esng = Speaker()

#esng.wpm = 130
#esng.pitch = 200
esng.voice = 'es'
esng.say("Bienvenidos a este nuevo tutorial hoy os enseñamos a descargar el GTA 6 yo no lo descargo porque lo tengo xdxdxd.")

## 4. Chatbots y cómo nos responde una IA.



### 4.1 Creando modelos en Rasa

Rasa es un framework de aprendizaje automático de código abierto para automatizar conversaciones basadas en texto y voz.

Rasa ayuda a crear asistentes contextuales capaces de mantener conversaciones por capas con muchas idas y venidas. Para que un humano tenga un intercambio significativo con un asistente contextual, el asistente debe ser capaz de utilizar el contexto para construir sobre las cosas que se discutieron previamente - Rasa le permite construir asistentes que pueden hacer esto de una manera escalable.

Para crear un modelo en Rasa invocaremos el siguiente comando.

```bash
rasa init
```

Con ello se crea un árbol parecido a este:

```
chatbot
|-- .rasa
|-- cache (Incluye todos los archivos temporales de entrenamiento de Rasa)
|-- actions
||-- actions.py
||-- __init__.py
|-- data
||-- nlu.yml
||-- rules.yml
||-- stories.yml
|-- models (Incluye los modelos en formato .tar.gz)
|-- tests
||-- test_stories.yml
|-- config.yml
|-- credentials.yml
|-- domain.yml
|-- endpoints.yml
```

### 4.2 Entrenamiento y pruebas

Rasa tiene como unidades básicas para entender la pregunta las intenciones (o intents), que son las ideas que se quieren transmitir. Así, cuando alguien saluda, lo puede decir de varias formas (*Hola, Buenos días, Hey ...*). Podremos controlar los ejemplos de las intenciones (y crear las nuestras propias) en `data/nlu.yml`

Para dar las respuestas, Rasa usa dos conceptos:

- **Declaraciones (o utterances)**: Son aquellas respuestas que sólo precisan de una serie de textos (Por ejemplo, si pedimos que salude, podemos decir que siempre responda con un Muy buenas, ¿qué tal?). Estas respuestas se declaran en la sección responses de domain.yml.
- **Acciones (o actions)**: Son aquellas respuestas más complejas que necesitan de información dinámica (como poner frases aleatorias o decir la hora). Estas respuestas se declaran en la sección actions de domain.yml, y se deben programar en actions/actions.py (se explicará cómo hacerlo en secciones posteriores).

Para poder saber cómo relacionar las preguntas y las respuestas, tenemos dos maneras. Por una parte, podemos hacer que sigan reglas de forma que
cada vez que se manifieste una intención, debemos siempre relacionarlo con una respuesta. Estas reglas se pueden establecer en el archivo data/rules.yml
Por otra parte, si queremos que sigan un flujo de conversación, podemos hacer historias de usuario, donde representamos una serie de intenciones
y sus respuestas en forma de acción o declaración, que podrán usarse para entrenar el modelo a base de ejemplos. Estas historias de usuario podemos tenerlas como parte del entrenamiento o como parte de la validación (que quedan aparte del entrenamiento para comprobar que lo aprendido se puede exportar a otros casos parecidos). Los casos de uso de entrenamiento se podrán introducir en data/stories.yml, y los de validación, en tests/test_stories.yml
Cuando se hagan cambios en el modelo, hay que volverlo a entrenar. Desde el archivo de configuración (config.yml) podemos hacer ajustes en las políticas de entrenamiento.

Para entrenar el modelo en Rasa invocaremos el siguiente comando; quien usando los casos de uso de entrenamiento como de validación, podrá aprender a entender las intenciones y dar así una respuesta.

```bash
rasa train
```

Para probarlo, invocaremos este otro comando.
```bash
rasa shell
```

### 4.3 Conectando el servidor de Rasa.

Para poner a Rasa en modo de servidor, podremos lanzar este comando:

```bash
rasa run
```

El puerto se puede configurar en `endpoints.yml`. 

Al configurar el puerto y lanzar el servidor podemos hacer peticiones a Rasa desde otros scripts, incluso otros PC. 

In [ ]:
import requests
sender = input("What is your name?\n")

bot_message = ""
while bot_message != "Adiós":
    message = input("What's your message?\n")

    print("Sending message now...")

    r = requests.post('http://localhost:5005/webhooks/rest/webhook', json={"sender": sender, "message": message})

    print(r.json())

    print("Bot says, ")
    for i in r.json():
        try:
            bot_message = i['text']
        except KeyError:
            try:
                bot_message = i['image']
            except KeyError:
                bot_message = ''
        print(f"{bot_message}")

### 4.4 Funciones custom

Las funciones custom se pueden lanzar usando:

```bash
rasa run actions
```

In [ ]:
# En el archivo chatbot/actions/actions.py ...

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import requests

class ActionTellTime(Action):
    """
    Clase para la acción de decir la hora
    """

    def name(self) -> Text:
        """
        Declaración de la acción
        @return string Nombre de la acción
        """
        return "action_tell_time"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        """
        Ejecución de la acción.
        Extrae el datetime de este instante y extrae la hora
        @utter_message Devuelve la hora en formato HH y MM (15:30 -> Son las 15 horas y 30 minutos)
        """
        dispatcher.utter_message(text=f"Son las \
            {datetime.datetime.now().strftime('%H horas y %M minutos')}")

        return []

### 4.5 Entidades

Las entidades son palabras que en su conjunto podemos asociar a una temática. Por ejemplo, podemos asociar como países palabras como España, Italia, República Checa ...

Nuestro chatbot no sabe asociar este tipo de palabras pero nosotros se lo podemos decir usando los ejemplos con `[ejemplo](tipo_entidad)`.

Así, por ejemplo:

```yaml
- intent: city_weather
  examples: |
    - qué tiempo hace en [granada](city)
    - cómo está el tiempo en [Málaga](city)
    - cómo está el clima en [madrid](city)
    - cómo está [Londres](city) hoy
    - dime qué tiempo hace en [parís](city)
    - dime el tiempo que hace en [Nueva York](city)
    - que tiempo hace en [granada](city)
    - dime el tiempo que hace en [sevilla](city)
    - dime como esta el clima en [malaga](city)
    - que tiempo hara en [barcelona](city)
    - que tiempo hara en [sevilla](city)
    - como esta el tiempo en [denia](city)
    - que clima hace en [granada](city)
    - que tiempo hara en [huelva](city)
```

Podemos usar las entidades capturadas en las acciones. Mira este ejemplo:

In [ ]:
# En actions.py...    
    
    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        city = next(tracker.get_latest_entity_values('city'),None)
       
        dispatcher.utter_message(text=f'Ahora mismo no te puedo decir mucho del tiempo\
                que hace en {city}. Consulta a tu meteorólogo.')

        return []

## 4.6 La respuesta por defecto

Hay ocasiones donde no sabemos qué responder porque puede ser algo estúpido o porque está fuera de tu dominio.

Por ejemplo, alguien puede preguntar algo como :

![SkullGirls Question](https://www.quartertothree.com/fp/wp-content/uploads/2014/07/riddles_in_the_skull.jpg)

Ante esta cuestión, Rasa "explota" una mijilla y pone una respuesta por defecto, pero podemos cambiar su comportamiento.

Para ello podemos crear una intención llamada `nlu_fallback`:

```yaml
- intent: nlu_fallback
  examples: |
    - amo a ver que dise
    - mefovjsfocjsobjdsvo
    - vkoskvpskvsspvpskvjisvhsihvsihvsiaa
    - aclmacm
    - asmadajcipadpdvaj
    - vsdpdkvpsdkvpskpvks
    - papapapapapapapappapapa
    - mamamamamam
    - adios
```

Con ello, podemos crear una regla o historia de usuario donde pongamos por intención el fallback y como respuesta lo que uno quiera.

## 5. Juntándolo todo

Una vez vistos los snippets de código y cómo funcionan, podremos seguir un flujo como el siguiente para ordenar los snippets como funciones y crear nuestro asistente.

![Diagrama Flujo](img/DiagramaIntuitivo.png)

## 5.1 Algunas cuestiones

- La voz de eSpeak es muy robótica.

Era la única que permitía funcionar en todas las plataformas. Hay otros TTS geniales como NanoTTS en el caso de Linux.

- ¿Y si lo queremos llevar a un sistema como una Raspi?

En versiones con poca RAM como la 3B no se puede ejecutar Rasa lamentablemente (aunque se le puede hace funcionar igualmente ajustando el programa como un Cliente/Servidor). En la 4 de 4GB ya se ha reportado un buen funcionamiento (lento, pero funciona)